<a href="https://colab.research.google.com/github/Carambolushka/Kaggle_competitions/blob/main/Synthtanic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

Идея!
Попробавать рандомный лес

Попробовать классификацию

Посмотреть на фичи

In [ ]:
!unzip '/content/drive/MyDrive/Data for Colab/Synthtanic/train.csv.zip' train.csv
!unzip '/content/drive/MyDrive/Data for Colab/Synthtanic/test.csv.zip' test.csv

Archive:  /content/drive/MyDrive/Data for Colab/Synthtanic/train.csv.zip
replace train.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: N
Archive:  /content/drive/MyDrive/Data for Colab/Synthtanic/test.csv.zip
  inflating: test.csv                


In [ ]:
train = pd.read_csv('/content/train.csv')
test = pd.read_csv('/content/test.csv')

In [ ]:
train.head(5)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,0,1,1,"Oconnor, Frankie",male,NaN,2,0,209245,27.14,C12239,S
1,1,0,3,"Bryan, Drew",male,NaN,0,0,27323,13.35,NaN,S
2,2,0,3,"Owens, Kenneth",male,0.33,1,2,CA 457703,71.29,NaN,S
3,3,0,3,"Kramer, James",male,19.00,0,0,A. 10866,13.04,NaN,S
4,4,1,3,"Bond, Michael",male,25.00,0,0,427635,7.76,NaN,S


Разделим сет на train, validation и test в отношении 80%/10%/10%

Теперь работа будет проводиться только с train_sample

Смотрим на нули

In [ ]:
def describe_dataset(dataset):
    print(f'всего {len(dataset.columns)} колонок')
    for i in range (len(dataset.columns)):
        print(f'Информация о колонке {dataset.columns[i]}')
        if len(dataset[dataset.columns[i]].unique()) > 10:
            print(f'В колонке {dataset.columns[i]} более 10 уникальных значений')
            print(f'Пример значений: {dataset[dataset.columns[i]][100]}')
            print(f'В колонке {dataset.columns[i]} - {sum(train[train.columns[i]].isna())} нулевых значений')
            print(f'В колонке {dataset.columns[i]} - {sum(train[train.columns[i]].notna())} ненулевых значений')
            print('\n', '*'*35)
        else:
            print(f'В колонке {dataset.columns[i]} следующие уникальные значения: {dataset[dataset.columns[i]].unique()}')
            print(f'В колонке {dataset.columns[i]} - {sum(train[train.columns[i]].isna())} нулевых значений')
            print(f'В колонке {dataset.columns[i]} - {sum(train[train.columns[i]].notna())} ненулевых значений')
            print('\n', '*'*35)
    return

In [ ]:
describe_dataset(train)

всего 12 колонок
Информация о колонке PassengerId
В колонке PassengerId более 10 уникальных значений
Пример значений: 100
В колонке PassengerId - 0 нулевых значений
В колонке PassengerId - 100000 ненулевых значений

 ***********************************
Информация о колонке Survived
В колонке Survived следующие уникальные значения: [1 0]
В колонке Survived - 0 нулевых значений
В колонке Survived - 100000 ненулевых значений

 ***********************************
Информация о колонке Pclass
В колонке Pclass следующие уникальные значения: [1 3 2]
В колонке Pclass - 0 нулевых значений
В колонке Pclass - 100000 ненулевых значений

 ***********************************
Информация о колонке Name
В колонке Name более 10 уникальных значений
Пример значений: Farber, Efrain
В колонке Name - 0 нулевых значений
В колонке Name - 100000 ненулевых значений

 ***********************************
Информация о колонке Sex
В колонке Sex следующие уникальные значения: ['male' 'female']
В колонке Sex - 0 нулевы

Из функции ниже видно, что колонок с нулевыми значениями - 5

Age

Ticket

Fare

Cabin

Embarked

In [ ]:
train.corr()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
PassengerId,1.000000,0.001956,-0.000893,-0.002654,0.001000,0.000411,-0.001036
Survived,0.001956,1.000000,-0.289723,0.103895,-0.007587,0.023352,0.187534
Pclass,-0.000893,-0.289723,1.000000,-0.259109,-0.051436,-0.041848,-0.417354
Age,-0.002654,0.103895,-0.259109,1.000000,-0.112459,-0.125535,0.109164
SibSp,0.001000,-0.007587,-0.051436,-0.112459,1.000000,0.213126,0.059358
Parch,0.000411,0.023352,-0.041848,-0.125535,0.213126,1.000000,0.061443
Fare,-0.001036,0.187534,-0.417354,0.109164,0.059358,0.061443,1.000000


Линейных зависимостей у переменных нет


In [ ]:
train[['Pclass', 'Fare']].groupby('Pclass').agg({ 'Fare':np.median}).reset_index()

,Pclass,Fare
0,1,63.58
1,2,22.72
2,3,10.96


## Работа с колонкой Fare

In [ ]:
train[train["Pclass"]==1]['Fare'].describe()

count    30275.000000
mean        92.653113
std         99.821850
min          3.380000
25%         27.890000
50%         63.580000
75%        116.875000
max        744.660000
Name: Fare, dtype: float64

In [ ]:
train[train["Pclass"]==2]['Fare'].describe()

count    28637.000000
mean        25.756003
std         34.849110
min          1.720000
25%         10.170000
50%         22.720000
75%         29.210000
max        613.430000
Name: Fare, dtype: float64

In [ ]:
train[train["Pclass"]==3]['Fare'].describe()

count    40954.000000
mean        20.618226
std         32.621763
min          0.680000
25%          8.540000
50%         10.960000
75%         25.770000
max        640.790000
Name: Fare, dtype: float64

In [ ]:
train.loc[(train["Pclass"]==1) & (train['Fare'].isna()), 'Fare']=92.3
test.loc[(test["Pclass"]==1) & (test['Fare'].isna()), 'Fare']=92.3

In [ ]:
train.loc[(train["Pclass"]==2) & (train['Fare'].isna()), 'Fare']=25.7
test.loc[(test["Pclass"]==2) & (test['Fare'].isna()), 'Fare']=25.7


In [ ]:
train.loc[(train["Pclass"]==3) & (train['Fare'].isna()), 'Fare']=20.6
test.loc[(test["Pclass"]==3) & (test['Fare'].isna()), 'Fare']=20.6

Проверить на нулевые значения колонку Fare

In [ ]:
train['Fare'].isna().sum()

0

## Работа с колонкой Age

Распределение по классу и по полу

In [ ]:
train[['Age', 'Pclass']].groupby('Pclass').aggregate({'Age':'mean'})

,Age
Pclass,
1,44.774464
2,38.631237
3,33.407362


In [ ]:
train[['Age', 'Sex']].groupby('Sex').aggregate({'Age':'mean'})

,Age
Sex,
female,40.044890
male,37.030884


In [ ]:
train[(train['Pclass']==1)  & (train['Sex']=='male')]['Age'].describe()

count    13751.000000
mean        42.254922
std         14.554998
min          0.080000
25%         32.000000
50%         44.000000
75%         53.000000
max         80.000000
Name: Age, dtype: float64

In [ ]:
train[(train['Pclass']==2)  & (train['Sex']=='male')]['Age'].describe()

count    12947.000000
mean        37.839926
std         17.234730
min          0.080000
25%         25.000000
50%         37.000000
75%         52.000000
max         85.000000
Name: Age, dtype: float64

In [ ]:
train[(train['Pclass']==3)  & (train['Sex']=='male')]['Age'].describe()

count    27509.000000
mean        34.038757
std         17.159494
min          0.080000
25%         23.000000
50%         31.000000
75%         48.000000
max         83.000000
Name: Age, dtype: float64

In [ ]:
train[(train['Pclass']==1)  & (train['Sex']=='female')]['Age'].describe()

count    15602.000000
mean        46.995091
std         14.480906
min          0.330000
25%         38.000000
50%         49.000000
75%         58.000000
max         83.000000
Name: Age, dtype: float64

In [ ]:
train[(train['Pclass']==2)  & (train['Sex']=='female')]['Age'].describe()

count    14784.000000
mean        39.324223
std         20.865713
min          0.080000
25%         24.000000
50%         40.000000
75%         57.000000
max         87.000000
Name: Age, dtype: float64

In [ ]:
train[(train['Pclass']==3)  & (train['Sex']=='female')]['Age'].describe()

count    12115.000000
mean        31.973681
std         21.252496
min          0.080000
25%         13.000000
50%         28.000000
75%         50.000000
max         85.000000
Name: Age, dtype: float64

In [ ]:
train.loc[(train['Pclass']==1) & (train['Sex']=='male') & (train['Age'].isna()), 'Age'] = 42
test.loc[(test['Pclass']==1) & (test['Sex']=='male') & (test['Age'].isna()), 'Age'] = 42

In [ ]:
train.loc[(train['Pclass']==2) & (train['Sex']=='male') & (train['Age'].isna()), 'Age'] = 38
test.loc[(test['Pclass']==2) & (test['Sex']=='male') & (test['Age'].isna()), 'Age'] = 38

In [ ]:
train.loc[(train['Pclass']==3) & (train['Sex']=='male') & (train['Age'].isna()), 'Age'] = 34
test.loc[(test['Pclass']==3) & (test['Sex']=='male') & (test['Age'].isna()), 'Age'] = 34

In [ ]:
train.loc[(train['Pclass']==1) & (train['Sex']=='female') & (train['Age'].isna()), 'Age'] = 47
test.loc[(test['Pclass']==1) & (test['Sex']=='female') & (test['Age'].isna()), 'Age'] = 47

In [ ]:
train.loc[(train['Pclass']==2) & (train['Sex']=='female') & (train['Age'].isna()), 'Age'] = 39
test.loc[(test['Pclass']==2) & (test['Sex']=='female') & (test['Age'].isna()), 'Age'] = 39

In [ ]:
train.loc[(train['Pclass']==3) & (train['Sex']=='female') & (train['Age'].isna()), 'Age'] = 32
test.loc[(test['Pclass']==3) & (test['Sex']=='female') & (test['Age'].isna()), 'Age'] = 32

In [ ]:
train[(train['Age'].isna())]

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked


## Теперь работа с колонкой Embarked

In [ ]:
train[(train['Embarked']=='S')]
test[(test['Embarked']=='S')]

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,100000,3,"Holliday, Daniel",male,19.0,0,0,24745,63.01,NaN,S
1,100001,3,"Nguyen, Lorraine",female,53.0,0,0,13264,5.81,NaN,S
3,100003,2,"Larsen, Eric",male,25.0,0,0,314011,12.93,NaN,S
6,100006,1,"Guerrero, Angel",female,48.0,1,2,PC 28237,221.94,B21146,S
7,100007,3,"Payton, Brian",male,13.0,0,0,42031,9.41,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...
99990,199990,3,"Leroy, Morton",male,18.0,1,1,C.A. 458349,55.09,NaN,S
99992,199992,3,"Sonsino, Paris",male,31.0,0,0,28965,10.07,NaN,S
99993,199993,3,"Leftwich, Don",male,45.0,0,0,42267,7.02,NaN,S
99996,199996,1,"Brown, Howard",male,59.0,1,0,13004,68.31,NaN,S


In [ ]:
embarked_class = train[['Embarked', 'Pclass']].groupby(['Embarked', 'Pclass']).agg({'Pclass':'count'}).rename(columns={'Embarked':'Embarked', 'Pclass':'Class', 'Pclass':'Count_class'}).reset_index()

In [ ]:
embarked_class['prevalence, %']=np.round(embarked_class.Count_class/sum(embarked_class.Count_class)*100, 1)

In [ ]:
embarked_class

,Embarked,Pclass,Count_class,"prevalence, %"
0,C,1,11843,11.9
1,C,2,6156,6.2
2,C,3,4188,4.2
3,Q,1,3751,3.8
4,Q,2,844,0.8
5,Q,3,829,0.8
6,S,1,14635,14.7
7,S,2,21609,21.7
8,S,3,35895,36.0


In [ ]:
train.loc[(train['Embarked'].isna()) & (train['Pclass']==3), 'Embarked'] = 'S'
test.loc[(test['Embarked'].isna()) & (test['Pclass']==3), 'Embarked'] = 'S'

In [ ]:
train.loc[(train['Embarked'].isna()) & (train['Pclass']==2), 'Embarked'] = 'Q'
test.loc[(test['Embarked'].isna()) & (test['Pclass']==2), 'Embarked'] = 'Q'

In [ ]:
train.loc[(train['Embarked'].isna()) & (train['Pclass']==1), 'Embarked'] = 'C'
test.loc[(test['Embarked'].isna()) & (test['Pclass']==1), 'Embarked'] = 'C'

In [ ]:
train[(train['Embarked'].isna())]

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked


Колонка Cabin в основном состоит из нулевых значений, поэтому заполнить ее какими-либо данными не представляется возможным. Ее убираем

In [ ]:
train=train.drop(labels='Cabin', axis=1)
test=test.drop(labels='Cabin', axis=1)

Также уберем колонку Ticket

In [ ]:
train=train.drop(labels='Ticket', axis=1)
test=test.drop(labels='Ticket', axis=1)

Еще раз смотрим на описание всего получившегося датасета

In [ ]:
describe_dataset(train)

всего 10 колонок
Информация о колонке PassengerId
В колонке PassengerId более 10 уникальных значений
Пример значений: 100
В колонке PassengerId - 0 нулевых значений
В колонке PassengerId - 100000 ненулевых значений

 ***********************************
Информация о колонке Survived
В колонке Survived следующие уникальные значения: [1 0]
В колонке Survived - 0 нулевых значений
В колонке Survived - 100000 ненулевых значений

 ***********************************
Информация о колонке Pclass
В колонке Pclass следующие уникальные значения: [1 3 2]
В колонке Pclass - 0 нулевых значений
В колонке Pclass - 100000 ненулевых значений

 ***********************************
Информация о колонке Name
В колонке Name более 10 уникальных значений
Пример значений: Farber, Efrain
В колонке Name - 0 нулевых значений
В колонке Name - 100000 ненулевых значений

 ***********************************
Информация о колонке Sex
В колонке Sex следующие уникальные значения: ['male' 'female']
В колонке Sex - 0 нулевы

Теперь нужно разобраться с текстовыми данными

С колонкой Name пока ничего не сделать

In [ ]:
train=train.drop(labels='Name', axis=1)
test=test.drop(labels='Name', axis=1)

In [ ]:
train.head()

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,1,1,male,42.00,2,0,27.14,S
1,1,0,3,male,34.00,0,0,13.35,S
2,2,0,3,male,0.33,1,2,71.29,S
3,3,0,3,male,19.00,0,0,13.04,S
4,4,1,3,male,25.00,0,0,7.76,S


Пол заменим 0 - male и 1 - female

In [ ]:
train.loc[(train['Sex']=='male'), 'Sex'] = 0
test.loc[(test['Sex']=='male'), 'Sex'] = 0

In [ ]:
train.loc[(train['Sex']=='female'), 'Sex'] = 1
test.loc[(test['Sex']=='female'), 'Sex'] = 1

Теперь с колонкой Embarked

In [ ]:
train.loc[(train['Embarked']=='C'), 'Embarked'] = 0
test.loc[(test['Embarked']=='C'), 'Embarked'] = 0

In [ ]:
train.loc[(train['Embarked']=='Q'), 'Embarked'] = 1
test.loc[(test['Embarked']=='Q'), 'Embarked'] = 1

In [ ]:
train.loc[(train['Embarked']=='S'), 'Embarked'] = 2
test.loc[(test['Embarked']=='S'), 'Embarked'] = 2

In [ ]:
describe_dataset(train)

всего 9 колонок
Информация о колонке PassengerId
В колонке PassengerId более 10 уникальных значений
Пример значений: 100
В колонке PassengerId - 0 нулевых значений
В колонке PassengerId - 100000 ненулевых значений

 ***********************************
Информация о колонке Survived
В колонке Survived следующие уникальные значения: [1 0]
В колонке Survived - 0 нулевых значений
В колонке Survived - 100000 ненулевых значений

 ***********************************
Информация о колонке Pclass
В колонке Pclass следующие уникальные значения: [1 3 2]
В колонке Pclass - 0 нулевых значений
В колонке Pclass - 100000 ненулевых значений

 ***********************************
Информация о колонке Sex
В колонке Sex следующие уникальные значения: [0 1]
В колонке Sex - 0 нулевых значений
В колонке Sex - 100000 ненулевых значений

 ***********************************
Информация о колонке Age
В колонке Age более 10 уникальных значений
Пример значений: 38.0
В колонке Age - 0 нулевых значений
В колонке Age - 

Разделяем наш датасет на тестовыйЮ трейновый и валидационный

In [ ]:
train_sample, valid_sample = np.split(train.sample(frac=1, random_state=42), [int(0.8*len(train))])

In [ ]:
X = train_sample.drop(labels='Survived', axis=1)
Y = train_sample['Survived']
X_valid = valid_sample.drop(labels='Survived', axis=1)
Y_valid = valid_sample['Survived']

In [ ]:
train_sample

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
75721,75721,1,2,1,57.0,0,0,8.32,0
80184,80184,1,3,0,26.0,0,0,6.17,2
19864,19864,0,3,1,31.0,0,0,7.47,2
76699,76699,1,2,1,41.0,0,0,8.16,2
92991,92991,1,2,1,26.0,0,0,61.50,2
...,...,...,...,...,...,...,...,...,...
6306,6306,1,1,1,52.0,0,0,29.58,2
51901,51901,1,3,0,26.0,0,0,23.31,2
6028,6028,0,3,0,30.0,4,0,12.49,2
58507,58507,0,1,0,52.0,1,2,73.88,2


In [ ]:
from sklearn.metrics import f1_score

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

In [ ]:
boosting = GradientBoostingClassifier(learning_rate=0.1, n_estimators=70, random_state=42).fit(X,Y)

In [ ]:
y_pred_boost = boosting.predict(X_valid)

In [ ]:
f1_score(Y_valid, y_pred_boost)

0.7347605436621362

In [ ]:
y_pred_test = boosting.predict(test)

In [ ]:
test_prediction=pd.DataFrame(y_pred_test)

In [ ]:
test_prediction['PassengerId'] = test['PassengerId']

In [ ]:
test_prediction = test_prediction[['PassengerId', 0]].rename(columns={0:'Survived'})

In [ ]:
test_prediction

,PassengerId,Survived
0,100000,0
1,100001,0
2,100002,1
3,100003,0
4,100004,1
...,...,...
99995,199995,1
99996,199996,0
99997,199997,0
99998,199998,1


In [ ]:
test_prediction.to_csv('answer.csv', index=False)